## Scikit learn to ONNX
- Neuromophic 연구 : ONNX <-> ML (Scikit learn)

In [18]:
# Iris Data Load 
# Train a model.

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112,) (38,)


In [20]:
# Logistic Regression Fit
from sklearn.linear_model import LogisticRegression

# 모델 생성 및 학습
clr = LogisticRegression()
clr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)

# onnx 모델 지정
with open("logreg_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    

699

In [63]:
print(type(onx))
onx

<class 'onnx.onnx_ml_pb2.ModelProto'>


ir_version: 4
producer_name: "skl2onnx"
producer_version: "1.6.1"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probability_tensor"
    name: "LinearClassifier"
    op_type: "LinearClassifier"
    attribute {
      name: "classlabels_ints"
      ints: 0
      ints: 1
      ints: 2
      type: INTS
    }
    attribute {
      name: "coefficients"
      floats: -0.4361514747142792
      floats: 0.835845410823822
      floats: -2.322659730911255
      floats: -0.9590612053871155
      floats: 0.5638760924339294
      floats: -0.3518039882183075
      floats: -0.16542649269104004
      floats: -0.8327014446258545
      floats: -0.12772460281848907
      floats: -0.48404139280319214
      floats: 2.488086223602295
      floats: 1.7917625904083252
      type: FLOATS
    }
    attribute {
      name: "intercepts"
      floats: 9.438634872436523
      floats: 1.6098077297210693
      floats: -11.048442840576172
    

In [52]:
# Compute the prediction with ONNX Runtime
# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession(path_or_bytes = "logreg_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

In [60]:
print(sess.get_inputs()[0].name)
print(sess.get_inputs()[0].shape)
print(sess.get_inputs()[0].type)
print('-----------------')
print(sess.get_outputs()[0].name)
print(sess.get_outputs()[0].shape)
print(sess.get_outputs()[0].type)

float_input
[None, 4]
tensor(float)
-----------------
output_label
[1]
tensor(int64)


In [70]:
sess.get_inputs()[0].shape

[None, 4]

In [53]:
pred_onx
label_name

array([1, 1, 0, 1, 1, 2, 0, 2, 0, 2, 2, 1, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1,
       2, 0, 0, 1, 1, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 2], dtype=int64)

'output_label'

In [14]:
# 
rt.__version__
rt.__author__

'1.0.0'

'Microsoft'

In [12]:
# function
rt.get_all_providers() # cpu, cuda, openvino, mkl-dnn(memory?)
rt.get_available_providers() 
rt.get_device() # cpu, gpu
rt.onnxruntime_validation.check_distro_info()

['TensorrtExecutionProvider',
 'CUDAExecutionProvider',
 'MKLDNNExecutionProvider',
 'NGRAPHExecutionProvider',
 'OpenVINOExecutionProvider',
 'NupharExecutionProvider',
 'BrainSliceExecutionProvider',
 'CPUExecutionProvider']

['CPUExecutionProvider']

'CPU'

In [ ]:
## 런타임 version
## 사용 device
## input name, shape, type
## output name, shape, type
## 

In [7]:
# Accuracy 평가지표
from sklearn.metrics import accuracy_score

# Predict에 대한 Accuracy Score
print('accuracy_score', float(accuracy_score(y_test, pred_onx))) 

accuracy_score 0.9473684210526315
